In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime as dt
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from war_functions.pecota_tables import *

from cluster_luck_functions.cluster_luck_hitting import *
from cluster_luck_functions.cluster_luck_pitching import *
from cluster_luck_functions.cluster_luck_combined import *

from daily_adjustments.active_rosters import *
from daily_adjustments.BP_WAR import *
from daily_adjustments.todays_game_info import *
from daily_adjustments.starting_rotations_WAR import *
from daily_adjustments.adjusted_war_today import *

from odds_and_other_projections import *

team_map = {
    'Giants' : 'San Francisco Giants',
    'Astros' : 'Houston Astros',
    'Brewers': 'Milwaukee Brewers',
    'Dodgers': 'Los Angeles Dodgers',
    'Rays' : 'Tampa Bay Rays',
    'Red Sox': 'Boston Red Sox',
    'White Sox': 'Chicago White Sox',
    'Padres': 'San Diego Padres',
    'Athletics': 'Oakland Athletics',
    'Yankees' : 'New York Yankees',
    'Mariners': 'Seattle Mariners',
    'Reds': 'Cincinnati Reds',
    'Blue Jays' : 'Toronto Blue Jays',
    'Mets' : 'New York Mets',
    'Phillies': 'Philadelphia Phillies',
    'Angels': 'Los Angeles Angels',
    'Braves': 'Atlanta Braves',
    'Cardinals': 'St. Louis Cardinals',
    'Tigers': 'Detroit Tigers',
    'Cubs': 'Chicago Cubs',
    'Indians': 'Cleveland Indians',
    'Guardians': 'Cleveland Guardians',
    'Nationals': 'Washington Nationals',
    'Rockies': 'Colorado Rockies',
    'Marlins': 'Miami Marlins',
    'Royals': 'Kansas City Royals',
    'Twins': 'Minnesota Twins',
    'Pirates': 'Pittsburgh Pirates',
    'Rangers': 'Texas Rangers',
    'Orioles': 'Baltimore Orioles',
    'Diamondbacks': 'Arizona Diamondbacks',
    'D-backs' : 'Arizona Diamondbacks'
}

In [2]:
# Retrieving necessary data

active_rosters = retrieve_all_active_rosters(file_name = None)
todays_games = retrieve_todays_games_info()
current_year_war = retrieve_current_year_WAR()
pt = load_combined_pecota_table()
odds = retrieve_odds()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST driver version for 100.0.4896
Get LATEST driver version for 100.0.4896
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_mac64.zip
Driver has been saved in cache [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/100.0.4896.60]


In [15]:
tables = pd.read_html('https://www.espn.com/mlb/stats/team')
runs_scored_table = pd.merge(tables[0], tables[1], left_on = tables[0].index, right_on = tables[1].index)
runs_scored_table.drop('key_0', axis = 1, inplace = True)
runs_scored_table = runs_scored_table[['Team', 'GP', 'R']]
runs_scored_table['Runs_162'] = runs_scored_table.R * (162.0/runs_scored_table.GP)

tables = pd.read_html(f'https://www.foxsports.com/mlb/team-stats?category=pitching&season={dt.date.today().year}&seasonType=reg')
runs_allowed_table = tables[1]
runs_allowed_table = runs_allowed_table.iloc[:,[1,2,13]]
runs_allowed_table.columns = ['Team', 'Games', 'Runs_Allowed']
runs_allowed_table['Runs_Allowed_162'] = runs_allowed_table.Runs_Allowed * (162.0/runs_allowed_table.Games)
runs_allowed_table['Team'] = runs_allowed_table.Team.apply(lambda x: team_map[x])

merged = pd.merge(runs_scored_table, runs_allowed_table, on = 'Team')
merged = merged[['Team', 'Games', 'Runs_162', 'Runs_Allowed_162']]

current_year_hitting = retrieve_historical_hitting_tables(2022, file_name = None)
hitting_reg = load_linear_regression("data/hitting_regression.pickle")
cluster_luck_hitting = calculate_predicted_cluster_luck_run_adjustment_hitting(hitting_reg, current_year_hitting)
cluster_luck_hitting = cluster_luck_hitting[['Team', 'GP', 'run_adjust']]
cluster_luck_hitting.columns = ['Team', 'Games', 'Offensive_Adjustment']

current_year_pitching = retrieve_historical_pitching_tables(dt.date.today().year, file_name = None)
pitching_reg = load_linear_regression('./data/pitching_regression.pickle')
cluster_luck_pitching = calculate_predicted_cluster_luck_run_adjustment_pitching(pitching_reg, current_year_pitching)

/Users/hsinger24/Desktop/Programming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/hsinger24/Desktop/Programming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Team,Games,Offensive_Adjustment
8,Blue Jays,14,2.724536
4,Angels,14,-3.411943
13,Braves,15,3.568238
15,Giants,14,-9.743104
2,Phillies,14,3.020446
3,Mets,15,-2.870135
17,Mariners,14,1.518463
27,Astros,13,0.274207
20,Athletics,15,-19.265177
29,Diamondbacks,14,3.596779


In [16]:
active_rosters

[{'team_name': 'Angels',
  'team_roster': [('Jo Adell', '666176'),
   ('Jaime Barria', '642545'),
   ('Archie Bradley', '605151'),
   ('Reid Detmers', '672282'),
   ('Matt Duffy', '622110'),
   ('Jimmy Herget', '623474'),
   ('Raisel Iglesias', '628452'),
   ('Michael Lorenzen', '547179'),
   ('Aaron Loup', '571901'),
   ('Brandon Marsh', '669016'),
   ('Mike Mayers', '594577'),
   ('Jack Mayfield', '608686'),
   ('Shohei Ohtani', '660271'),
   ('Oliver Ortega', '661383'),
   ('Anthony Rendon', '543685'),
   ('Patrick Sandoval', '663776'),
   ('Max Stassi', '545358'),
   ('Jose Suarez', '660761'),
   ('Kurt Suzuki', '435559'),
   ('Noah Syndergaard', '592789'),
   ('Ryan Tepera', '572193'),
   ('Mike Trout', '545361'),
   ('Andrew Velazquez', '623205'),
   ('Tyler Wade', '642180'),
   ('Jared Walsh', '665120'),
   ('Andrew Wantz', '681806'),
   ('Taylor Ward', '621493'),
   ('Austin Warren', '681810')]},
 {'team_name': 'Diamondbacks',
  'team_roster': [('Sergio Alcantara', '642727'),
 

In [39]:
starting_rotations, failed_to_find_pitchers = retrieve_starting_rotations_WAR(pt, current_year_war)

/Users/hsinger24/Desktop/Baseball-Betting/daily_adjustments/starting_rotations_WAR.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starting_pitchers['WAR_proj'] = 0
/Users/hsinger24/Desktop/Baseball-Betting/daily_adjustments/starting_rotations_WAR.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starting_pitchers['WAR'] = 0
/Users/hsinger24/Desktop/Baseball-Betting/daily_adjustments/starting_rotations_WAR.py:119: FutureWarning: elementwise comparison failed; returning scalar instead, but in the f

In [49]:
sp_adjustments = calculate_sp_adjustment(todays_games, starting_rotations, pt, frac_season = 0.08)

All names failed for       Unnamed: 0.1    bpid      name name_wo_a name_alt_1  name_alt_2  \
3208          3208  108959  Sam Long  Sam Long        NaN         NaN   

      name_alt_3  name_alt_4  
3208         NaN         NaN   Pecota WAR is: 0.3


/Users/hsinger24/Desktop/Baseball-Betting/daily_adjustments/adjusted_war_today.py:50: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if sp_home in names[col].values:
/Users/hsinger24/Desktop/Baseball-Betting/daily_adjustments/adjusted_war_today.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if sp_away in names[col].values:


In [45]:
overall_war_predictions_preseason = pd.read_csv('data/overall_war_predictions_preseason.csv')
active_roster_war, failed_to_find_players = calculate_active_roster_war_table(active_rosters, overall_war_predictions_preseason, current_year_war, pt, 2022, 0.08)

/Users/hsinger24/Desktop/Baseball-Betting/daily_adjustments/adjusted_war_today.py:227: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if player_name in names[col].values:


2022